In [12]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as f

In [3]:
spark = SparkSession.builder.master('local').appName('app').getOrCreate()
trade_common = spark.read.parquet("output_dir/partition=T")
trade = trade_common.select("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb", "arrival_tm", "trade_pr")


In [4]:
trade.createOrReplaceTempView('trade_v')

In [5]:
sql1 = """
select

"""

In [6]:
sample_data = [ ['aaa', 'aaa', 1],
                ['aaa', 'bbb', 3],
                ['aaa', 'ccc', 2],  
                ['bbb', 'ddd', 3],
                ['bbb', 'eee', 2],
                ['bbb', 'fff', 1]
              ]

In [7]:
rdd = spark.sparkContext.parallelize(sample_data)
df = rdd.toDF()

In [8]:
df.show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|aaa|aaa|  1|
|aaa|bbb|  3|
|aaa|ccc|  2|
|bbb|ddd|  3|
|bbb|eee|  2|
|bbb|fff|  1|
+---+---+---+



In [13]:
w = Window.partitionBy("_1")

In [19]:
df.withColumn('mm', f.max('_3').over(w)).filter(f.col('mm') == f.col('_3')).drop(f.col('mm')).show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|aaa|bbb|  3|
|bbb|ddd|  3|
+---+---+---+



In [20]:
df.withColumn('mm', f.max('_3').over(w)).filter(f.col('mm') == df['_3']).drop(f.col('mm')).show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|aaa|bbb|  3|
|bbb|ddd|  3|
+---+---+---+



In [9]:
df.createOrReplaceTempView('df_v')

In [10]:
spark.sql("""
select _1, _2 as p, _3,
row_number() OVER (Partition by _1 order by _3 desc) as q
from df_v
""").filter(f.col('q') == 1).show()

+---+---+---+---+
| _1|  p| _3|  q|
+---+---+---+---+
|aaa|bbb|  3|  1|
|bbb|ddd|  3|  1|
+---+---+---+---+



In [11]:
spark.sql("""
select _1, _2 as p, _3,
max(_3) OVER (Partition by _1) as q
from df_v
""").filter(f.col('q') == f.col('_3')).show()

+---+---+---+---+
| _1|  p| _3|  q|
+---+---+---+---+
|aaa|bbb|  3|  3|
|bbb|ddd|  3|  3|
+---+---+---+---+

